In [1]:
!pip install faiss-cpu sentence-transformers transformers gradio accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 47.0 MB/s eta 0:00:00


In [2]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr


In [3]:
# Embedding modeli
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Örnek veriler (bunları sonradan PDF veya DB verileriyle değiştirebiliriz)
documents = [
    "RAG, bilgi tabanındaki verileri çekerek yanıt üretimini güçlendirir.",
    "FAISS, Facebook tarafından geliştirilen hızlı vektör arama kütüphanesidir.",
    "Python, yapay zeka projelerinde sıklıkla kullanılır.",
    "Llama 3 modeli, yüksek kaliteli metin üretimi için tasarlanmıştır.",
    "Mistral, açık kaynaklı büyük dil modelleri arasında güçlü bir seçenektir."
]

# Embedding ve FAISS index
embeddings = embedder.encode(documents)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def retrieve(query, top_k=2):
    query_vec = embedder.encode([query])
    distances, indices = index.search(query_vec, top_k)
    return [documents[i] for i in indices[0]]

def rag_generate(query):
    retrieved_docs = retrieve(query)
    context = " ".join(retrieved_docs)
    # Adjust prompt format for Mistral Instruct models
    prompt = f"<s>[INST] Kontekst: {context}\nSoru: {query} [/INST]"
    response = generator(prompt, max_new_tokens=150, do_sample=True, temperature=0.7)
    # Extract the generated text after the assistant's turn
    generated_text = response[0]["generated_text"]
    # Find the start of the assistant's response
    assistant_start = generated_text.find("[/INST]") + len("[/INST]")
    return generated_text[assistant_start:].strip()

In [ ]:
def chatbot_interface(message, history):
    answer = rag_generate(message)
    return answer

iface = gr.ChatInterface(fn=chatbot_interface, title="🧠 RAG Chatbot (FAISS + Llama/Mistral)")
iface.launch(debug=True)


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a30eb9774cbf98651c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
